"How can we build a prediction system to detect electric car component failures before they occur?"

## Importing needed libraries

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder


## Dataset Handling

In [2]:
df = pd.read_csv('updated_pollution_dataset.csv')
print("Dataset Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())

Dataset Shape: (5000, 10)

Missing Values:
 Temperature                      0
Humidity                         0
PM2.5                            0
PM10                             0
NO2                              0
SO2                              0
CO                               0
Proximity_to_Industrial_Areas    0
Population_Density               0
Air Quality                      0
dtype: int64


In [3]:
# Handle missing values if any
df = df.dropna()

In [4]:
df = pd.read_csv('updated_pollution_dataset.csv')
print("Columns:", df.columns.tolist())

Columns: ['Temperature', 'Humidity', 'PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Proximity_to_Industrial_Areas', 'Population_Density', 'Air Quality']


## Dense Layer

In [12]:
class Dense_layer:
    """
    This class is used to define the dense layer in Neural Networks. 
    This included forward and backward propagation.
    """
    
    def __init__(self, n_inputs, n_neurons):
        """
        n_inputs: Number of inputs.
        n_neurons: Number of neurons in the layer

        Weights are defined with random values.
        Biases are defined as zeros.
        """
        self.weights = np.random.rand(n_inputs, n_neurons)  # randomly initialized weights
        self.biases = np.zeros((1, n_neurons)) # biases intialized as zeros

    def forward_propagation(self, input_layer):
        self.input_layer = input_layer
        self.output_layer = np.dot(input_layer, self.weights) + self.biases
        return self.output_layer
    
    def backward_propagation(self, output_error, learning_rate):
        self.d_weights = np.dot(self.input_layer.T, output_error) / self.input_layer.shape[0]
        self.d_biases = np.sum(output_error, axis=0, keepdims=True) / self.input_layer.shape[0]
        self.d_inputs = np.dot(output_error, self.weights.T)

        # Update weights and biases
        self.weights -= learning_rate * self.d_weights
        self.biases -= learning_rate * self.d_biases

        return self.d_inputs

## Sigmoid - Activation Function

In [13]:
class Sigmoid:
    """ 
    This class represents the sigmoid activation function.
    """
    def __init__(self):
        pass
    
    def forward_propagation(self, input):
        self.inputs = input
        self.output = 1 / (1 + np.exp(-input))
        return self.output
    
    def backward_propagation(self, output_error):
        return output_error * (self.output * (1 - self.output))

## Relu Activation Function

In [14]:
class Relu:
    """ 
    This class is for define Rectified Linear Unit (Relu) activation funciton.
    """
    def __init__(self):
        pass
    
    def forward_propagation(self, input):
        self.input = input
        self.output = np.maximum(0, input)
        return self.output
    
    def backward_propagation(self, output_error):
        return output_error * (self.output > 0).astype(float)

## Softmax - Activation Function

In [15]:
class Softmax:
    """ 
    This class is for define Softmax activation funciton.
    """
    def __init__(self):
        pass
    
    def forward_propagation(self, input):
        self.input = input
        ex = np.exp(input - np.max(input, axis=1, keepdims=True))
        self.output = ex / np.sum(ex, axis=1, keepdims=True)
        return self.output
    
    def backward_propagation(self, output_error):
        return output_error

## Dropout

In [16]:
class Dropout:
    def __init__(self, dropout_probabality):
        self.dropout_probabality = dropout_probabality
        self.mask = None

    def forward_propagation(self, input):
        if self.dropout_probabality < 1.0:
            self.mask = (np.random.rand(*input.shape) >
                         self.dropout_probabality) / (1 - self.dropout_probabality)

            return input * self.mask
        return input

    def backward_propagation(self, output_error):
        if self.dropout_probabality < 1.0:
            return output_error * self.mask
        return output_error

## Neural Network

In [17]:
class Neural_network:
   def __init__(self):
       self.layers = []

   def add_layer(self, layer, activation_func=None, dropout=None):
       self.layers.append(
           {"layer": layer, "activation_func": activation_func, "dropout": dropout})

   def forward_propagation(self, X):
       self.input = X
       for layer_details in self.layers:
           X = layer_details["layer"].forward_propagation(X)
           if layer_details["activation_func"] is not None:
               X = layer_details["activation_func"].forward_propagation(X)
           if layer_details["dropout"] is not None:
               X = layer_details["dropout"].forward_propagation(X)
       self.output = X
       return self.output

   def backward_propagation(self, output_error, learning_rate):
       for layer_details in reversed(self.layers):
            if layer_details["dropout"]:
                output_error = layer_details["dropout"].backward_propagation(
                    output_error)
            if layer_details["activation_func"]:
                output_error = layer_details["activation_func"].backward_propagation(
                    output_error)
            output_error = layer_details["layer"].backward_propagation(
               output_error, learning_rate)

   def train(self, X, y, learning_rate, epochs, batch_size=32):
        for epoch in range(epochs):
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i+batch_size]
                y_batch = y[i:i+batch_size]
                predictions = self.forward_propagation(X_batch)
                #categorical cross-entropy loss
                loss = - \
                    np.mean(np.sum(y_batch * np.log(predictions + 1e-7), axis=1))
                output = predictions - y_batch
                self.backward_propagation(output, learning_rate)

            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

   def predict(self, X):
        prediction = self.forward_propagation(X)
        return np.argmax(prediction, axis=1)

## Data Preprocessing, Balancing, and Scaling for Air Quality Prediction

Loading the dataset and feature selection

In [18]:
df = pd.read_csv("updated_pollution_dataset.csv")

X = df.drop("Air Quality", axis=1).values
y = df["Air Quality"].values

One-hot encoding of the target variable

In [21]:
encoder = OneHotEncoder()
y_one_hot = encoder.fit_transform(y.reshape(-1, 1)).toarray()

Balancing the dataset using SMOTE

In [22]:
# Balance classes using SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, np.argmax(y_one_hot, axis=1))

In [23]:
# Convert y_balanced back to numeric values (already integers after SMOTE)
y_balanced = np.array(y_balanced).reshape(-1, 1)

In [24]:
# One-hot encode the balanced target
y_balanced = encoder.fit_transform(y_balanced).toarray()

Data Split and Scaling features 

In [25]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)



In [26]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

 Neural Network Initialization

In [31]:

nn = Neural_network()
nn.add_layer(Dense_layer(X_train.shape[1], 16), Relu(),Dropout(0.05))
nn.add_layer(Dense_layer(16, 8), Relu(),Dropout(0.05))
nn.add_layer(Dense_layer(8, 4), Softmax())



In [32]:
# Train the network with batch training
nn.train(X_train, y_train, epochs=300, learning_rate=0.01, batch_size=64)

Epoch 1/300, Loss: 1.2197
Epoch 2/300, Loss: 0.9105
Epoch 3/300, Loss: 1.0735
Epoch 4/300, Loss: 1.2782
Epoch 5/300, Loss: 1.0294
Epoch 6/300, Loss: 1.0236
Epoch 7/300, Loss: 0.9380
Epoch 8/300, Loss: 0.8681
Epoch 9/300, Loss: 0.9196
Epoch 10/300, Loss: 0.8745
Epoch 11/300, Loss: 0.8606
Epoch 12/300, Loss: 0.8406
Epoch 13/300, Loss: 0.8201
Epoch 14/300, Loss: 0.8074
Epoch 15/300, Loss: 0.8294
Epoch 16/300, Loss: 0.7793
Epoch 17/300, Loss: 0.7262
Epoch 18/300, Loss: 0.7451
Epoch 19/300, Loss: 0.7593
Epoch 20/300, Loss: 0.7327
Epoch 21/300, Loss: 0.7015
Epoch 22/300, Loss: 0.7074
Epoch 23/300, Loss: 0.6891
Epoch 24/300, Loss: 0.7201
Epoch 25/300, Loss: 0.6923
Epoch 26/300, Loss: 0.6379
Epoch 27/300, Loss: 0.6723
Epoch 28/300, Loss: 0.6494
Epoch 29/300, Loss: 0.6927
Epoch 30/300, Loss: 0.6213
Epoch 31/300, Loss: 0.6205
Epoch 32/300, Loss: 0.6071
Epoch 33/300, Loss: 0.6400
Epoch 34/300, Loss: 0.6182
Epoch 35/300, Loss: 0.6511
Epoch 36/300, Loss: 0.6085
Epoch 37/300, Loss: 0.6341
Epoch 38/3

Network Evaluation 

In [33]:

predictions = nn.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.9250


## Implement Optimizer 

In [34]:
class SGD_Momentum:
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.velocity = {}
    
    def initialize(self, params):
        # Initialize velocity for each parameter
        for layer_idx in range(len(params)):
            self.velocity[f'weights_{layer_idx}'] = np.zeros_like(params[layer_idx]['weights'])
            self.velocity[f'biases_{layer_idx}'] = np.zeros_like(params[layer_idx]['biases'])
    
    def update(self, params, gradients):
        for layer_idx in range(len(params)):
            # Update weights using momentum
            self.velocity[f'weights_{layer_idx}'] = (self.momentum * self.velocity[f'weights_{layer_idx}'] - 
                                                   self.learning_rate * gradients[layer_idx]['d_weights'])
            self.velocity[f'biases_{layer_idx}'] = (self.momentum * self.velocity[f'biases_{layer_idx}'] - 
                                                  self.learning_rate * gradients[layer_idx]['d_biases'])
            
            params[layer_idx]['weights'] += self.velocity[f'weights_{layer_idx}']
            params[layer_idx]['biases'] += self.velocity[f'biases_{layer_idx}']

In [35]:
class MiniBatchSGD:
    def __init__(self, learning_rate=0.01, batch_size=32):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
    
    def initialize(self, params):
        pass  # No initialization needed for basic SGD
    
    def update(self, params, gradients):
        for layer_idx in range(len(params)):
            params[layer_idx]['weights'] -= self.learning_rate * gradients[layer_idx]['d_weights']
            params[layer_idx]['biases'] -= self.learning_rate * gradients[layer_idx]['d_biases']

## Testing Different Optimizers
